# Working with Azure ML SDK

In [ ]:
import os
import json

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute

In [ ]:
aml = {
    'subscription_id': 'ENTER_YOUR_SUBSCRIPTION_ID_HERE',
    'resource_group': 'ENTER_YOUR_RESOURCE_GROUP_NAME_HERE',
    'workspace_name': 'ENTER_YOUR_WORKSPACE_NAME_HERE',
}
ws = Workspace(aml['subscription_id'], aml['resource_group'], aml['workspace_name'])
ws.write_config()
config = json.load(open("/".join(['.azureml', 'config.json']), 'r'))
print(config)

In [ ]:
experiment_name = 'Demo-Pytorch-Lightning-on-MNIST'
exp = Experiment(ws, experiment_name)

In [ ]:
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print(compute_target.get_status().serialize())


In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('Demo-Environment-for-Pytorch-Lightning-on-MNIST')
cd = CondaDependencies.create(
    pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'],
    conda_packages=['pytorch-lightning']
)

env.python.conda_dependencies = cd

env.register(workspace=ws)

In [ ]:
from azureml.core import ScriptRunConfig

args = ["--max-epochs", str(1), "--adadelta-lr", str(0.5)]

src = ScriptRunConfig(source_directory=os.path.join(os.getcwd(), "scripts"),
                      script='train.py', 
                      arguments=args,
                      compute_target=compute_target,
                      environment=env)

In [ ]:
run = exp.submit(config=src)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()